In [1]:
import warnings
warnings.filterwarnings("ignore")
from joblib import parallel_backend
import multiprocessing
n_cores = multiprocessing.cpu_count()
import csv

# MILP1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")

# Danh sách các file dữ liệu
file_names = [
    'australia.txt',
    "australia_noise_label_feature.txt",
    "australia_outlier.txt",
    'australia_both_noise_outlier.txt'
]
print('Model: MILP1')
results = []
# Tạo danh sách để lưu kết quả
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
        df = pd.read_csv(file_name, header= None, sep = ' ')
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
        df = pd.read_csv(file_name, header= None)
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
        df = pd.read_csv(file_name, header= None)
    else:
        noise_type = 'Noise + Outlier'
        df = pd.read_csv(file_name, header= None)
    

    X = df.iloc[:,:14].values
    y = df.iloc[:,14].values
    y = np.where(y == 0, -1, y)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    # Các tham số của mô hình
    c = [1.0] * X.shape[1]
    l = [-2] * X.shape[1]  # Giới hạn dưới của trọng số wj
    u = [2] * X.shape[1]   # Giới hạn trên của trọng số wj
    C_values = [2**i for i in range(-3,6)]

    # Áp dụng 10-fold cross-validation để tìm giá trị tối ưu cho B
    best_B = None
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0

    # Danh sách các giá trị B cần kiểm tra
    n = X.shape[1]
    B_values = [i for i in range(1, n+1)]

    # for C in C_values:
    with parallel_backend('loky', n_jobs=n_cores):
        for B in B_values:
            for C in C_values:
                kf = KFold(n_splits=10, shuffle=True,random_state=42)
                cv_accuracies = []
                cv_AUC = []
                cv_times = []
                
                for train_index, val_index in kf.split(X):
                    start_time = time.time()
                    X_cv_train, X_cv_val = X[train_index], X[val_index]
                    y_cv_train, y_cv_val = y[train_index], y[val_index]
                    
                    X_cv_train = scaler.fit_transform(X_cv_train)
                    X_cv_val = scaler.transform(X_cv_val)

                    # Khởi tạo mô hình
                    opt_mod = Model(name='MILP1')

                    # Số lượng mẫu và số lượng thuộc tính
                    m, n = X_cv_train.shape

                    # Các biến quyết định
                    w = opt_mod.continuous_var_list(n, name='w')
                    b = opt_mod.continuous_var(name='b')
                    v = opt_mod.binary_var_list(n, name='v')
                    xi = opt_mod.continuous_var_list(m, lb=0, name='xi')
                    z = opt_mod.continuous_var_list(n,lb =0, name = 'z')

                  
                    opt_mod.minimize(opt_mod.sum(xi[i] for i in range(m)))

                    # Ràng buộc
                    for i in range(m):
                        opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i,j] for j in range(n)) + b) >= 1 - xi[i])
                    for j in range(n):
                        opt_mod.add_constraint(w[j] <= v[j] * u[j])
                        opt_mod.add_constraint(w[j] >= l[j] * v[j])
                        opt_mod.add_constraint(w[j] >= -z[j])
                        opt_mod.add_constraint(w[j] <= z[j])
                
                    opt_mod.add_constraint(opt_mod.sum(c[j] * v[j] for j in range(n)) <= B)
         
                    solution = opt_mod.solve()

                    end_time = time.time()
                    fold_time = end_time - start_time
                    cv_times.append(fold_time)

                    if solution:
                        w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                        b_opt = solution.get_value('b')
                        v_opt = np.array([solution.get_value(f'v_{j}') for j in range(n)])
                        y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                        cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                        cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                        cv_accuracies.append(cv_accuracy)
                        cv_AUC.append(cv_auc)
                    

            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

            if mean_cv_AUC > best_mean_cv_AUC:
                best_B = B
                best_C = C
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_b = b_opt
                best_v = v_opt
        print(f'Best B value: {best_B}')
        print(f'Best C value: {best_C}')
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]


        
        print(f'v: {best_v}')
        print(f'Features selected: {selected_features}')
        print(f'Number of selected features: {len(selected_features)}')
        print('-' * 100)

        result = {
            'Model': 'MILP1',  
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C
        }
        results.append(result)

output_file = 'model_results_australia.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: MILP1

Xử lý file: D:\Optimal-Robust-Feature-Selection-For-Support-Vector-Machines-With-Pinball-Loss\Dataset\Dataset\australia.txt
Not noise


KeyboardInterrupt: 

# Pin-FS-SVM

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")

# Danh sách các file dữ liệu
file_names = [
    "australia.txt",
    "australia_noise_label_feature.txt",
    "australia_outlier.txt",
    'australia_both_noise_outlier.txt'
]
print('Model: MILP1')
results = []
# Tạo danh sách để lưu kết quả
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
        df = pd.read_csv(file_name, header= None, sep = ' ')
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
        df = pd.read_csv(file_name, header= None)
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
        df = pd.read_csv(file_name, header= None)
    else:
        noise_type = 'Noise + Outlier'
        df = pd.read_csv(file_name, header= None)
    

    X = df.iloc[:,:14].values
    y = df.iloc[:,14].values
    y = np.where(y == 0, -1, y)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    # Các tham số của mô hình
    l = [-2] * X.shape[1]
    u = [2] * X.shape[1]
    C_values = [2**i for i in range(-3, 6)]
    tau_values = [0.1, 0.5, 1]

    best_B = None
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0
    no_improvement_count = 0

    n = X.shape[1]
    B_values = [i for i in range(1, n+1)]

    start_file_time = time.time()

    with parallel_backend('loky', n_jobs=n_cores):
        for tau in tau_values:
            for C in C_values:
                for B in B_values:
                    kf = KFold(n_splits=10, shuffle=True, random_state=42)
                    cv_accuracies = []
                    cv_AUC = []
                    cv_times = []

                    for train_index, val_index in kf.split(X):
                        start_time = time.time()
                        X_cv_train, X_cv_val = X[train_index], X[val_index]
                        y_cv_train, y_cv_val = y[train_index], y[val_index]

                        # Chuẩn hóa dữ liệu
                        X_cv_train = scaler.fit_transform(X_cv_train)
                        X_cv_val = scaler.transform(X_cv_val)

                        # Khởi tạo mô hình
                        opt_mod = Model(name='Pin-FS-SVM')
                        m, n = X_cv_train.shape

                        # Các biến quyết định
                        w = opt_mod.continuous_var_list(n, name='w')
                        b = opt_mod.continuous_var(name='b')
                        v = opt_mod.binary_var_list(n, name='v')
                        xi = opt_mod.continuous_var_list(m, lb=0, name='xi')
                        z = opt_mod.continuous_var_list(n, lb=0, name='z')

                        # Hàm mục tiêu
                        opt_mod.minimize(opt_mod.sum(z[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                        # Ràng buộc phân loại
                        for i in range(m):
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) <= 1 + xi[i] * (1/tau))

                        # Ràng buộc tổng các đặc trưng được chọn
                        opt_mod.add_constraint(opt_mod.sum(v[j] for j in range(n)) <= B)

                        
                        for j in range(n):
                            opt_mod.add_constraint(w[j] <= v[j] * u[j])
                            opt_mod.add_constraint(w[j] >= l[j] * v[j])
                            opt_mod.add_constraint(w[j] <= z[j])
                            opt_mod.add_constraint(w[j] >= -z[j])

                        # Giải mô hình
                        solution = opt_mod.solve()

                        end_time = time.time()
                        fold_time = end_time - start_time
                        cv_times.append(fold_time)

                        if solution:
                            w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                            b_opt = solution.get_value('b')
                            v_opt = np.array([solution.get_value(f'v_{j}') for j in range(n)])
                            y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                            cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                            cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                            cv_accuracies.append(cv_accuracy)
                            cv_AUC.append(cv_auc)
                        else:
                            print("Solution not found for fold.")

                    # Đánh giá trung bình trên tập huấn luyện bằng 10-fold cross-validation với mỗi giá trị B
                    mean_cv_accuracy = np.mean(cv_accuracies)
                    mean_cv_AUC = np.mean(cv_AUC)
                    mean_time_per_fold = np.mean(cv_times)

                    # Kiểm tra điều kiện early stopping
                    if mean_cv_AUC > best_mean_cv_AUC:
                        best_B = B
                        best_C = C
                        best_tau = tau
                        best_mean_cv_accuracy = mean_cv_accuracy
                        best_mean_cv_AUC = mean_cv_AUC
                        best_mean_time_per_fold = mean_time_per_fold
                        best_w = w_opt
                        best_b = b_opt
                        best_v = v_opt
                    

    print(f'Best B value: {best_B}')
    print(f'Best C: {best_C}')
    print(f'Best tau: {best_tau}')
    print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
    print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
    print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

    selected_features = [j + 1 for j in range(n) if best_w[j] != 0]

    print(f'v: {best_v}')
    print(f'Features selected: {selected_features}')
    print(f'Number of selected features: {len(selected_features)}')
    print('-' * 100)
    result = {
            'Model': 'Pin-FS-SVM',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C,
            'tau': best_tau
        }
    results.append(result)

output_file = 'model_results_australia.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C', 'tau']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: MILP1

Xử lý file: australia.txt
Not noise


Best B value: 1
Best C: 0.125
Best tau: 0.1
Best Accuracy on 10-CV: 0.855072463768116
Best AUC on 10-CV:  0.8578515743847543
Average time per fold: 0.13 seconds
v: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Features selected: [8]
Number of selected features: 1
----------------------------------------------------------------------------------------------------

Xử lý file: australia_noise_label_feature.txt
Noise
Best B value: 1
Best C: 0.125
Best tau: 0.1
Best Accuracy on 10-CV: 0.8144927536231885
Best AUC on 10-CV:  0.8162975120891316
Average time per fold: 0.15 seconds
v: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Features selected: [8]
Number of selected features: 1
----------------------------------------------------------------------------------------------------

Xử lý file: australia_outlier.txt
Outlier
Best B value: 1
Best C: 0.125
Best tau: 0.1
Best Accuracy on 10-CV: 0.808695652173913
Best AUC on 10-CV:  0.8239706182015019
Average time per fold: 0.13 seconds
v: [0. 0. 0. 0. 

# Pinball loss - SVM


In [17]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")

# Danh sách các file dữ liệu
file_names = [
    "australia.txt",
    "australia_noise_label_feature.txt",
    "australia_outlier.txt",
    'australia_both_noise_outlier.txt'
]
print('Model: Pinball loss')
results = []
# Tạo danh sách để lưu kết quả
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
        df = pd.read_csv(file_name, header= None, sep = ' ')
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
        df = pd.read_csv(file_name, header= None)
    else:
        print('Outlier')
        noise_type = 'Outlier'
        df = pd.read_csv(file_name, header= None)
    

    X = df.iloc[:,:14].values
    y = df.iloc[:,14].values
    y = np.where(y == 0, -1, y)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

   
    C_values = [2**i for i in range(-3,6)]
    tau_values = [0.1, 0.5, 1]
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0
    best_C = None
    n = X.shape[1]
    with parallel_backend('loky', n_jobs=n_cores):
        for C in C_values:
            for tau in tau_values:
                kf = KFold(n_splits=10, shuffle=True, random_state=42)
                cv_accuracies = []
                cv_AUC = []
                cv_times = []
                
                for train_index, val_index in kf.split(X):
                    start_time = time.time()
                    X_cv_train, X_cv_val = X[train_index], X[val_index]
                    y_cv_train, y_cv_val = y[train_index], y[val_index]
                    
                    X_cv_train = scaler.fit_transform(X_cv_train)
                    X_cv_val = scaler.transform(X_cv_val)

                    # Khởi tạo mô hình
                    opt_mod = Model(name='Pinball loss')

                    # Số lượng mẫu và số lượng thuộc tính
                    m, n = X_cv_train.shape

                    # Các biến quyết định
                    w = opt_mod.continuous_var_list(n, name='w')
                    b = opt_mod.continuous_var(name='b')
                    v = opt_mod.binary_var_list(n, name='v')
                    xi = opt_mod.continuous_var_list(m, lb=0, name='xi')
                    
                    
                    # Hàm mục tiêu
                    opt_mod.minimize(0.5 * opt_mod.sum(w[j] ** 2 for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                    # Ràng buộc phân loại
                    for i in range(m):
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                    for i in range(m):
                            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) <= 1 + xi[i]*(1/tau))

                    
                    # Giải mô hình
                    solution = opt_mod.solve()

                    end_time = time.time()
                    fold_time = end_time - start_time
                    cv_times.append(fold_time)

                    if solution:
                        w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                        b_opt = solution.get_value('b')
                        y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                        cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                        cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                        cv_accuracies.append(cv_accuracy)
                        cv_AUC.append(cv_auc)
                    else:
                        print("Solution not found for fold.")

            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

       
            if mean_cv_accuracy > best_mean_cv_accuracy:
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_C = C
                best_b = b_opt
                best_tau = tau
 
        print('Best C: ', best_C)
        print('Best tau: ', best_tau)
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]

        print(f'Number of selected features: {len(list(best_w))}')
        print('-' * 100)
        result = {
            'Model': 'Pin-SVM',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C,
            'tau' : best_tau
        }
        results.append(result)

# Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'model_results_australia.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C', 'tau']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: Pinball loss

Xử lý file: australia.txt
Not noise
Best C:  0.125
Best tau:  1
Best Accuracy on 10-CV: 0.855072463768116
Best AUC on 10-CV:  0.8578515743847543
Average time per fold: 0.12 seconds
Number of selected features: 14
----------------------------------------------------------------------------------------------------

Xử lý file: australia_noise_label_feature.txt
Noise
Best C:  0.125
Best tau:  1
Best Accuracy on 10-CV: 0.8144927536231885
Best AUC on 10-CV:  0.8162975120891316
Average time per fold: 0.12 seconds
Number of selected features: 14
----------------------------------------------------------------------------------------------------

Xử lý file: australia_outlier.txt
Outlier
Best C:  0.125
Best tau:  1
Best Accuracy on 10-CV: 0.808695652173913
Best AUC on 10-CV:  0.8239706182015019
Average time per fold: 0.12 seconds
Number of selected features: 14
----------------------------------------------------------------------------------------------------

Xử lý file:

# L1 - SVM

In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")

# Danh sách các file dữ liệu
file_names = [
    "australia.txt",
    "australia_noise_label_feature.txt",
    "australia_outlier.txt",
    'australia_both_noise_outlier.txt'
]
print('Model: L1-SVM')
results = []
# Tạo danh sách để lưu kết quả
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
        df = pd.read_csv(file_name, header= None, sep = ' ')
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
        df = pd.read_csv(file_name, header= None)
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
        df = pd.read_csv(file_name, header= None)
    else:
        noise_type = 'Noise + Outlier'
        df = pd.read_csv(file_name, header= None)
    

    X = df.iloc[:,:14].values
    y = df.iloc[:,14].values
    y = np.where(y == 0, -1, y)
    scaler = StandardScaler()

    # Các tham số của mô hình
    C_values = [2**i for i in range(-3, 6)]

    best_C = None
    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0

    # Áp dụng 10-fold cross-validation để tìm giá trị tối ưu cho C
    with parallel_backend('loky', n_jobs=n_cores):
        for C in C_values:
            kf = KFold(n_splits=10, shuffle=True, random_state=42)
            cv_accuracies = []
            cv_AUC = []
            cv_times = []

            for train_index, val_index in kf.split(X):
                start_time = time.time()

                X_cv_train, X_cv_val = X[train_index], X[val_index]
                y_cv_train, y_cv_val = y[train_index], y[val_index]
                
                X_cv_train = scaler.fit_transform(X_cv_train)
                X_cv_val = scaler.transform(X_cv_val)

                # Khởi tạo mô hình
                opt_mod = Model(name='L1-SVM')

                # Số lượng mẫu và số lượng thuộc tính
                m, n = X_cv_train.shape

                # Các biến quyết định
                w = opt_mod.continuous_var_list(n, name='w')
                b = opt_mod.continuous_var(name='b')
                v = opt_mod.continuous_var_list(n, name='v', lb=0)
                xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

                # Hàm mục tiêu
                opt_mod.minimize(opt_mod.sum(v[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                # Ràng buộc
                for i in range(m):
                    opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                for j in range(n):
                    opt_mod.add_constraint(w[j] <= v[j])
                    opt_mod.add_constraint(-v[j] <= w[j])

                # Giải mô hình
                solution = opt_mod.solve()

                end_time = time.time()
                fold_time = end_time - start_time
                cv_times.append(fold_time)

                if solution:
                    w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                    b_opt = solution.get_value('b')
                    y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                    cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                    cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                    cv_accuracies.append(cv_accuracy)
                    cv_AUC.append(cv_auc)

            # Đánh giá trung bình trên tập huấn luyện bằng 10-fold cross-validation với mỗi giá trị C
            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

            # Cập nhật giá trị C tối ưu
            if mean_cv_accuracy > best_mean_cv_accuracy:
                best_C = C
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_b = b_opt
        print(f'Best B value: {best_B}')
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]


        print(f'v: {best_v}')
        print(f'Features selected: {selected_features}')
        print(f'Number of selected features: {len(selected_features)}')
        print('-' * 100)
        result = {
            'Model': 'L1-SVM',  
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C
        }
        results.append(result)

# Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'model_results_australia.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: L1-SVM

Xử lý file: australia.txt
Not noise
Best B value: 1
Best Accuracy on 10-CV: 0.855072463768116
Best AUC on 10-CV:  0.8578515743847543
Average time per fold: 0.06 seconds
v: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Features selected: [4, 5, 6, 7, 8, 9, 10, 14]
Number of selected features: 8
----------------------------------------------------------------------------------------------------

Xử lý file: australia_noise_label_feature.txt
Noise
Best B value: 1
Best Accuracy on 10-CV: 0.8144927536231885
Best AUC on 10-CV:  0.8162975120891316
Average time per fold: 0.06 seconds
v: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Features selected: [5, 7, 8, 9, 10, 11, 14]
Number of selected features: 7
----------------------------------------------------------------------------------------------------

Xử lý file: australia_outlier.txt
Outlier
Best B value: 1
Best Accuracy on 10-CV: 0.808695652173913
Best AUC on 10-CV:  0.8239706182015019
Average time per fold: 0.06 seconds
v: [0

# L2 - SVM

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import time
import warnings
warnings.filterwarnings("ignore")

# Danh sách các file dữ liệu
file_names = [
    "australia.txt",
    "australia_noise_label_feature.txt",
    "australia_outlier.txt",
    'australia_both_noise_outlier.txt'
]
print('Model: L2-SVM')
results = []
# Tạo danh sách để lưu kết quả
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
        df = pd.read_csv(file_name, header= None, sep = ' ')
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
        df = pd.read_csv(file_name, header= None)
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
        df = pd.read_csv(file_name, header= None)
    else:
        noise_type = 'Noise + Outlier'
        df = pd.read_csv(file_name, header= None)
    

    X = df.iloc[:,:14].values
    y = df.iloc[:,14].values
    y = np.where(y == 0, -1, y)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()

    

    best_mean_cv_accuracy = 0
    best_mean_cv_AUC = 0
    best_C = None
    C_values = [2**i for i in range(-3, 6)]

    with parallel_backend('loky', n_jobs=n_cores):
        for C in C_values:
            kf = KFold(n_splits=10, shuffle=True, random_state=42)
            cv_accuracies = []
            cv_AUC = []
            cv_times = []

            for train_index, val_index in kf.split(X):
                start_time = time.time()

                X_cv_train, X_cv_val = X[train_index], X[val_index]
                y_cv_train, y_cv_val = y[train_index], y[val_index]
                
                X_cv_train = scaler.fit_transform(X_cv_train)
                X_cv_val = scaler.transform(X_cv_val)

                # Khởi tạo mô hình
                opt_mod = Model(name='L2-SVM')

                # Số lượng mẫu và số lượng thuộc tính
                m, n = X_cv_train.shape

                # Các biến quyết định
                w = opt_mod.continuous_var_list(n, name='w')
                b = opt_mod.continuous_var(name='b')
                xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

                # Hàm mục tiêu
                opt_mod.minimize(0.5 * opt_mod.sum(w[j] ** 2 for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

                # Ràng buộc
                for i in range(m):
                    opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train[i, j] for j in range(n)) + b) >= 1 - xi[i])
                
                # Giải mô hình
                solution = opt_mod.solve()

                end_time = time.time()
                fold_time = end_time - start_time
                cv_times.append(fold_time)

                if solution:
                    w_opt = np.array([solution.get_value(f'w_{j}') for j in range(n)])
                    b_opt = solution.get_value('b')
                    y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
                    cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
                    cv_auc = roc_auc_score(y_cv_val, y_cv_pred)
                    cv_accuracies.append(cv_accuracy)
                    cv_AUC.append(cv_auc)
                    

            mean_cv_accuracy = np.mean(cv_accuracies)
            mean_cv_AUC = np.mean(cv_AUC)
            mean_time_per_fold = np.mean(cv_times)

            # Cập nhật giá trị B tối ưu
            if mean_cv_accuracy > best_mean_cv_accuracy:
                best_C = C
                best_mean_cv_accuracy = mean_cv_accuracy
                best_mean_cv_AUC = mean_cv_AUC
                best_mean_time_per_fold = mean_time_per_fold
                best_w = w_opt
                best_b = b_opt
        print(f'Best Accuracy on 10-CV: {best_mean_cv_accuracy}')
        print(f'Best AUC on 10-CV:  {best_mean_cv_AUC}')
        print(f'Average time per fold: {best_mean_time_per_fold:.2f} seconds')

        selected_features = [j + 1 for j in range(n) if best_w[j] != 0]

        print(f'Features selected: {selected_features}')
        print(f'Number of selected features: {len(selected_features)}')
        print('-' * 100)
        result = {
            'Model': 'L2-SVM',  # Thay đổi tên mô hình tương ứng
            'Type of model': noise_type,
            'Accuracy': best_mean_cv_accuracy,
            'AUC': best_mean_cv_AUC,
            'Time': best_mean_time_per_fold,
            'Features selected': ', '.join(map(str, selected_features)),
            'Number of features': len(selected_features),
            'C': best_C
        }
        results.append(result)

# Sau khi chạy xong tất cả các mô hình, thêm đoạn code sau để in ra file CSV
output_file = 'v2_model_results_australia.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")


Model: MILP1

Xử lý file: australia.txt
Not noise
Best B value: 1
Best Accuracy on 10-CV: 0.855072463768116
Best AUC on 10-CV:  0.8578515743847543
Average time per fold: 0.10 seconds
v: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Features selected: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Number of selected features: 14
----------------------------------------------------------------------------------------------------

Xử lý file: australia_noise_label_feature.txt
Noise
Best B value: 1
Best Accuracy on 10-CV: 0.8144927536231885
Best AUC on 10-CV:  0.8162975120891316
Average time per fold: 0.10 seconds
v: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Features selected: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Number of selected features: 14
----------------------------------------------------------------------------------------------------

Xử lý file: australia_outlier.txt
Outlier
Best B value: 1
Best Accuracy on 10-CV: 0.808695652173913
Best AUC on 10-CV:  0.8239706182015

# Fisher-SVM

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from docplex.mp.model import Model
import time

# Danh sách các file dữ liệu
file_names = [
    "australia.txt",
    "australia_noise_label_feature.txt",
    "australia_outlier.txt",
    'australia_both_noise_outlier.txt'
]

# Tạo danh sách để lưu kết quả
results = []
print('Model: Fisher - SVM')
def calculate_f_score(X, y):
    pos_indices = y == 1
    neg_indices = y == -1
    
    X_positive = X[pos_indices]
    X_negative = X[neg_indices]
    
    # So luong mau duong am
    n_positive = X_positive.shape[0]
    n_negative = X_negative.shape[0]
    
    pos_mean = np.mean(X_positive, axis=0)
    neg_mean = np.mean(X_negative, axis=0)
    total_mean = np.mean(X, axis=0)
    
    variance_positive = np.sum((X_positive - pos_mean)**2, axis=0) / (n_positive - 1)
    variance_negative = np.sum((X_negative - neg_mean)**2, axis=0) / (n_negative - 1)
    
    numerator = (pos_mean - total_mean)**2 + (neg_mean - total_mean)**2
    denominator = variance_positive + variance_negative
    
    f_scores = numerator / denominator
    return f_scores

def svm_train_qp(X, y, C, lambda_val=1e-5):
        # Khởi tạo mô hình
        opt_mod = Model(name='L1-SVM')

        # Số lượng mẫu và số lượng thuộc tính
        m, n = X.shape

        # Các biến quyết định
        w = opt_mod.continuous_var_list(n, name='w')
        b = opt_mod.continuous_var(name='b')
        v = opt_mod.continuous_var_list(n, name='v', lb=0)
        xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

        # Hàm mục tiêu
        opt_mod.minimize(opt_mod.sum(v[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

        # Ràng buộc
        for i in range(m):
            opt_mod.add_constraint(y[i] * (opt_mod.sum(w[j] * X[i, j] for j in range(n)) + b) >= 1 - xi[i])
        for j in range(n):
            opt_mod.add_constraint(w[j] <= v[j])
            opt_mod.add_constraint(-v[j] <= w[j])
        
        solution = opt_mod.solve()
        if solution:
            w_opt = np.array([solution.get_value(w[j]) for j in range(n)])
            b_opt = solution.get_value(b)
            return w_opt, b_opt
        return None, None

def feature_selection_with_f_score_cv(X, y, thresholds, C_values):
    f_scores = calculate_f_score(X, y)
    best_accuracy = 0 
    best_auc = 0
    best_threshold = None
    best_error = float('inf')
    avg_fold_times = []
    for C in C_values:
        for threshold in thresholds:
            selected_features = f_scores >= threshold
            X_selected = X[:, selected_features]
            
            errors = []
            for _ in range(5):
                X_train, X_valid, y_train, y_valid = train_test_split(X_selected, y, test_size=0.2)
                
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_valid_scaled = scaler.transform(X_valid)
                
                w, b = svm_train_qp(X_train_scaled, y_train, C=C)
                
                if w is None:
                    continue
                
                y_pred = np.sign(np.dot(X_valid_scaled, w) + b)
                error = 1 - accuracy_score(y_valid, y_pred)
                errors.append(error)
            
            avg_error = np.mean(errors)
            if avg_error < best_error:
                best_error = avg_error
                best_threshold = threshold
                
        selected_features = f_scores >= best_threshold
        X_subset = X[:, selected_features]
        
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        cv_accuracies = []
        cv_aucs = []
        times = []
            
        for train_idx, val_idx in kf.split(X_subset):
            # Dữ liệu huấn luyện và kiểm tra
            start_time = time.time()
            X_train, X_val = X_subset[train_idx], X_subset[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            # Chuẩn hóa trên tập huấn luyện và áp dụng vào tập kiểm tra
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)

            # Tính b_opt dựa trên tập huấn luyện
            w_opt, b_opt = svm_train_qp(X_train, y_train, C=C)
            if w_opt is None:
                continue
            y_pred = np.sign(np.dot(X_val, w_opt) + b_opt)
            cv_accuracies.append(accuracy_score(y_val, y_pred))
            cv_aucs.append(roc_auc_score(y_val, y_pred))
            end_time = time.time()
            times.append(end_time - start_time)
        
        mean_accuracy = np.mean(cv_accuracies)
        mean_auc = np.mean(cv_aucs)
        mean_time = np.mean(times)
        avg_fold_times.append(mean_time)
        
        
        if mean_auc > best_auc:
            best_accuracy = mean_accuracy
            best_auc = mean_auc
            best_threshold = threshold
            best_C = C
        
    # Lưu lại accuracy và số lượng đặc trưng nếu tốt hơn kết quả tốt nhất
    avg_time_per_fold = np.mean(avg_fold_times)
    return selected_features, best_accuracy, best_auc, best_C, best_threshold, avg_time_per_fold

# Kết quả
C_values = [2**i for i in range(-3, 6)]

# Danh sách các file dữ liệu
# Tạo danh sách để lưu kết quả
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
        df = pd.read_csv(file_name, header= None, sep = ' ')
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
        df = pd.read_csv(file_name, header= None)
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
        df = pd.read_csv(file_name, header= None)
    else:
        noise_type = 'Noise + Outlier'
        df = pd.read_csv(file_name, header= None)
    
    # df = pd.read_csv(file_name, header= None)

    X = df.iloc[:,:14].values
    y = df.iloc[:,14].values
    y = np.where(y == 0, -1, y)
    
    thresholds = np.percentile(calculate_f_score(X, y), [25, 50, 75])
    selected_features, best_accuracy, best_auc, best_C, best_threshold, avg_time_per_fold = feature_selection_with_f_score_cv(X, y, thresholds, C_values)
    
    print("Accuracy tốt nhất:", best_accuracy)
    print("Các đặc trưng tốt nhất:", np.where(selected_features)[0] + 1)
    print('So luong featuers', len(np.where(selected_features)[0]))
    print('AUC tốt nhất:', best_auc)
    print('C tốt nhất:', best_C)
    print('Threshold tốt nhất:', best_threshold)
    print('Average time per fold:', avg_time_per_fold)
    result ={
        'Model': 'Fisher - SVM',
        'Type of model': noise_type,
        'Accuracy': best_accuracy,
        'AUC': best_auc,
        'Time': avg_time_per_fold,
        'Features selected': ', '.join(map(str, np.where(selected_features)[0] + 1)),
        'Number of features': len(np.where(selected_features)[0]),
        'C': best_C
        
    }
    results.append(result)
output_file = 'model_results_australia.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào file {output_file}")

Model: Filter2 - SVM

Xử lý file: australia.txt
Not noise
Accuracy tốt nhất: 0.855072463768116
Các đặc trưng tốt nhất: [ 5  8  9 10]
So luong featuers 4
AUC tốt nhất: 0.8578515743847543
C tốt nhất: 0.125
Threshold tốt nhất: 0.1524113450180166
Average time per fold: 0.043700287077162

Xử lý file: australia_noise_label_feature.txt
Noise
Accuracy tốt nhất: 0.8144927536231885
Các đặc trưng tốt nhất: [ 3  5  6  7  8  9 10]
So luong featuers 7
AUC tốt nhất: 0.8162975120891316
C tốt nhất: 0.125
Threshold tốt nhất: 0.03278592529416553
Average time per fold: 0.0447323375278049

Xử lý file: australia_outlier.txt
Outlier
Accuracy tốt nhất: 0.808695652173913
Các đặc trưng tốt nhất: [ 2  3  4  5  6  7  8  9 10 12]
So luong featuers 10
AUC tốt nhất: 0.8239706182015019
C tốt nhất: 0.125
Threshold tốt nhất: 0.006128740907158606
Average time per fold: 0.048928525712754994

Xử lý file: australia_both_noise_outlier.txt
Accuracy tốt nhất: 0.7782608695652173
Các đặc trưng tốt nhất: [ 3  5  6  7  8  9 10]
S

# RFE - SVM

In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from docplex.mp.model import Model
import multiprocessing
import time
import csv

n_cores = multiprocessing.cpu_count()
print(f"Số lõi CPU có sẵn: {n_cores}")

# Danh sách các file dữ liệu
file_names = [
    "australia.txt",
    "australia_noise_label_feature.txt",
    "australia_outlier.txt",
    'australia_both_noise_outlier.txt'
]
print('Model: RFE2-SVM')
results = []
# Tạo danh sách để lưu kết quả
for iteration, file_name in enumerate(file_names):
    
    print(f"\nXử lý file: {file_name}")
    if iteration == 0: 
        print('Not noise')
        noise_type = 'Not noise'
        df = pd.read_csv(file_name, header= None, sep = ' ')
    elif iteration == 1:
        print('Noise')
        noise_type = 'Noise'
        df = pd.read_csv(file_name, header= None)
    elif iteration == 2:
        print('Outlier')
        noise_type = 'Outlier'
        df = pd.read_csv(file_name, header= None)
    else:
        noise_type = 'Noise + Outlier'
        df = pd.read_csv(file_name, header= None)
    

    X = df.iloc[:,:14].values
    y = df.iloc[:,14].values
    y = np.where(y == 0, -1, y)

    # SVM huấn luyện sử dụng Quadratic Programming (QP)
    def svm_train_qp(X, y, C, lambda_val=1e-5):
        # Khởi tạo mô hình
        opt_mod = Model(name='L1-SVM')

        # Số lượng mẫu và số lượng thuộc tính
        m, n = X.shape

        # Các biến quyết định
        w = opt_mod.continuous_var_list(n, name='w')
        b = opt_mod.continuous_var(name='b')
        v = opt_mod.continuous_var_list(n, name='v', lb=0)
        xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

        # Hàm mục tiêu
        opt_mod.minimize(opt_mod.sum(v[j] for j in range(n)) + C * opt_mod.sum(xi[i] for i in range(m)))

        # Ràng buộc
        for i in range(m):
            opt_mod.add_constraint(y[i] * (opt_mod.sum(w[j] * X[i, j] for j in range(n)) + b) >= 1 - xi[i])
        for j in range(n):
            opt_mod.add_constraint(w[j] <= v[j])
            opt_mod.add_constraint(-v[j] <= w[j])
        
        solution = opt_mod.solve()
        if solution:
            w_opt = np.array([solution.get_value(w[j]) for j in range(n)])
            b_opt = solution.get_value(b)
            return w_opt, b_opt
        return None, None

    # Hàm SVM-RFE với tìm kiếm Cross-Validation và tối ưu C
    def svm_rfe_with_cv(X, y, C_values):
        s = list(range(X.shape[1]))  # Các đặc trưng ban đầu
        best_result = {'accuracy': 0, 'auc': 0, 'num_features': len(s), 'features': s[:], 'C': None, 'avg_time_per_fold': 0}
        
        for C in C_values:
            s_temp = s[:]  # Bắt đầu với tất cả các đặc trưng ban đầu trong mỗi vòng lặp của C
            while len(s_temp) > 1:
                # Lấy các đặc trưng hiện tại trong s_temp
                X_subset = X[:, s_temp]
                w_opt, b_opt = svm_train_qp(X_subset, y, C)
                if w_opt is None:
                    break

                # Đánh giá mô hình bằng Cross-Validation
                cv_accuracies, cv_aucs = [], []
                times = []
                kf = KFold(n_splits=10, shuffle=True, random_state=42)
                for train_idx, val_idx in kf.split(X_subset):
                    start_time = time.time()
                    X_train, X_val = X_subset[train_idx], X_subset[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    # Chuẩn hóa
                    scaler = StandardScaler()
                    X_train = scaler.fit_transform(X_train)
                    X_val = scaler.transform(X_val)

                    w_opt, b_opt = svm_train_qp(X_train, y_train, C)
                    if w_opt is None:
                        continue

                    y_pred = np.sign(np.dot(X_val, w_opt) + b_opt)
                    cv_accuracies.append(accuracy_score(y_val, y_pred))
                    cv_aucs.append(roc_auc_score(y_val, y_pred))
                    end_time = time.time()
                    times.append(end_time - start_time)

                mean_accuracy = np.mean(cv_accuracies)
                mean_auc = np.mean(cv_aucs)
                mean_time_per_fold = np.mean(times)

                # Cập nhật kết quả tốt nhất cho giá trị C hiện tại
                if mean_auc > best_result['auc']:
                    best_result.update({
                        'accuracy': mean_accuracy,
                        'auc': mean_auc,
                        'num_features': len(s_temp),
                        'features': s_temp[:],
                        'C': C,
                        'avg_time_per_fold': mean_time_per_fold
                    })

                # Loại bỏ đặc trưng ít quan trọng nhất
                num_to_remove = len(s_temp) // 2
                least_important_feature = np.argsort(w_opt ** 2)[:num_to_remove]
                s_temp = [s_temp[i] for i in range(len(s_temp)) if i not in least_important_feature]

        return best_result

    # Danh sách các giá trị C để tìm kiếm
    C_values = [2 ** i for i in range(-3, 6)]
    
    # Gọi hàm SVM-RFE với tìm kiếm giá trị C tối ưu
    best_result = svm_rfe_with_cv(X, y, C_values)

    # In kết quả
    print("Số lượng đặc trưng tốt nhất:", best_result['num_features'])
    print("Accuracy tốt nhất:", best_result['accuracy'])
    print("Các đặc trưng tốt nhất:", list( i + 1 for i in best_result['features'] ))
    print("AUC tốt nhất:", best_result['auc'])
    print("Giá trị C tốt nhất:", best_result['C'])
    print("Thời gian trung bình mỗi fold:", best_result['avg_time_per_fold'])
    
    results.append({
        'Model': 'RFE2-SVM',
        'Type of model': noise_type,
        'Accuracy': best_result['accuracy'],
        'AUC': best_result['auc'],
        'Time': best_result['avg_time_per_fold'],
        'Features selected': ', '.join(map(str, list( i + 1 for i in best_result['features'] ))),
        'Number of features': best_result['num_features'],
        'C': best_result['C']
    })

output_file = 'v2_model_results_australia.csv'
fieldnames = ['Model', 'Type of model', 'Accuracy', 'AUC', 'Time', 'Features selected', 'Number of features', 'C']

with open(output_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)
print(f"Kết quả đã được lưu vào file {output_file}")

Số lõi CPU có sẵn: 20
Model: RFE2-SVM

Xử lý file: australia.txt
Not noise
Số lượng đặc trưng tốt nhất: 14
Accuracy tốt nhất: 0.855072463768116
Các đặc trưng tốt nhất: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
AUC tốt nhất: 0.8578515743847543
Giá trị C tốt nhất: 0.125
Thời gian trung bình mỗi fold: 0.058407831192016604

Xử lý file: australia_noise_label_feature.txt
Noise
Số lượng đặc trưng tốt nhất: 14
Accuracy tốt nhất: 0.8144927536231885
Các đặc trưng tốt nhất: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
AUC tốt nhất: 0.8162975120891316
Giá trị C tốt nhất: 0.125
Thời gian trung bình mỗi fold: 0.05828502178192139

Xử lý file: australia_outlier.txt
Outlier
Số lượng đặc trưng tốt nhất: 14
Accuracy tốt nhất: 0.808695652173913
Các đặc trưng tốt nhất: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
AUC tốt nhất: 0.8239706182015019
Giá trị C tốt nhất: 0.125
Thời gian trung bình mỗi fold: 0.05000572204589844

Xử lý file: australia_both_noise_outlier.txt
Số lượng đặc trưng tốt nhất: